In [3]:
import os
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Function to load and preprocess the dataset
def load_dataset(data_dir):
    data = []
    labels = []
    for label in os.listdir(data_dir):
        path = os.path.join(data_dir, label)
        for img in os.listdir(path):
            img_path = os.path.join(path, img)
            image = cv2.imread(img_path)
            image = cv2.resize(image, (224, 224))  # Adjust size according to your model
            image = image / 255.0  # Normalize pixel values
            data.append(image)
            labels.append(label)
    return np.array(data), np.array(labels)

# Load and preprocess the dataset
data_dir = 'C:\\Users\\88016\\Desktop\\Mask_Detection\\data'
X, y = load_dataset(data_dir)

# Convert labels to binary format
y = (y == 'with_mask').astype(int)

# Define the model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

# Save the trained model
model.save('mask_detection_model.h5')


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\88016\\Desktop\\data'

In [ ]:
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing import image

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the trained mask classification model
model = keras.models.load_model('C:\\Users\\88016\\Desktop\\Mask_Detection\\mask_detection_model.h5')

# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Extract the face region
        roi = frame[y:y+h, x:x+w]

        # Preprocess the face image for the model
        roi = cv2.resize(roi, (224, 224))  # Adjust the size according to your model's input size
        roi = image.img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)

        # Make predictions using the model
        predictions = model.predict(roi)

        # Check if the predicted value is greater than 0.5
        label = 'With Mask' if predictions[0][0] > 0.5 else 'Without Mask'

        # Draw bounding box and label on the frame
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Mask Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 29ms/step


In [ ]:
1/1 [==============================] - 0s 95ms/step
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
Cell In[1], line 37
     35 # Make predictions using your model
     36 predictions = model.predict(roi)
---> 37 if predictions[0][0] > predictions[0][1]:
     38     label = 'With Mask'
     39 else:

IndexError: index 1 is out of bounds for axis 0 with size 1